In [1]:
import os
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point


from scipy.ndimage import maximum_filter as maxf2D
from scipy.ndimage import minimum_filter as minf2D
from scipy.ndimage import convolve as conf2D
# https://stackoverflow.com/questions/43966350/getting-the-maximum-in-each-rolling-window-of-a-2d-numpy-array

import numpy as np
#import matplotlib.pyplot as plt
#import rioxarray as rioxr
#import geopandas as gpd
#import pandas as pd
import rasterio
#import shapely
#import os

import pystac_client
import planetary_computer as pc

import lidar_sampling_functions as lsf

In [2]:
# lsf.save_min_max_diff_rasters(lidar_rast_r, os.getcwd())
# lsf.save_avg_rasters(lidar_rast_r, os.getcwd())

In [3]:
# accesing Azure storage using pystac client
URL = "https://planetarycomputer.microsoft.com/api/stac/v1"
catalog = pystac_client.Client.open(URL)

#campus point 2018 naip scene
itemid = 'ca_m_3411934_sw_11_060_20180722_20190209'
search = catalog.search(
    collections=["naip"],
    ids = itemid
)
item = list(search.get_items())[0]
# sign and open item
href = pc.sign(item.assets["image"].href)
ds = rasterio.open(href)
ds.crs

CRS.from_epsg(26911)

In [4]:
fp = '/home/jovyan/msai4earth-esa/iceplant_detection/data_sampling/points_from_naip_images/campus_lagoon_points/campus_lagoon_points_2018.csv'
pts = lsf.geodataframe_from_csv(fp, ds.crs)

/srv/conda/envs/notebook/lib/python3.8/site-packages/pandas/core/dtypes/cast.py:122: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  arr = construct_1d_object_array_from_listlike(values)


In [5]:
pts.head(3)

,iceplant,r,g,b,nir,year,month,day,naip_id,polygon_id,geometry
0,1,93,88,93,141,2018,7,22,ca_m_3411934_sw_11_060_20180722_20190209,0,POINT (238343.430 3810848.805)
1,1,85,85,89,137,2018,7,22,ca_m_3411934_sw_11_060_20180722_20190209,0,POINT (238343.547 3810846.093)
2,1,89,85,91,133,2018,7,22,ca_m_3411934_sw_11_060_20180722_20190209,0,POINT (238344.864 3810845.535)


In [6]:
fp = os.path.join(os.getcwd(),'3mlidar_over_campuspoint_naip_scene.tif')
lidar_rast_r = rasterio.open(fp)

In [7]:
pts_xy = lsf.pts_for_lidar_sampling(pts, lidar_rast_r.crs)

In [8]:
lidar_samples = lsf.sample_raster(pts_xy, lidar_rast_r)

In [9]:
fp = os.path.join(os.getcwd(),'lidar_maxs.tif')
maxs_rast_r = rasterio.open(fp)
max_samples = lsf.sample_raster(pts_xy, maxs_rast_r)

In [10]:
fp = os.path.join(os.getcwd(),'lidar_mins.tif')
mins_rast_r = rasterio.open(fp)
min_samples = lsf.sample_raster(pts_xy, mins_rast_r)

In [11]:
fp = os.path.join(os.getcwd(),'lidar_avgs.tif')
avg_rast_r = rasterio.open(fp)
avg_samples = lsf.sample_raster(pts_xy, avg_rast_r)

In [12]:
pts['lidar'] = lidar_samples
pts['max_lidar_w']= max_samples
pts['min_lidar_w'] = min_samples
pts['min_max_diff'] = pts.max_lidar_w - pts.min_lidar_w
pts['avg_lidar_w'] = avg_samples
pts.head(3)

,iceplant,r,g,b,nir,year,month,day,naip_id,polygon_id,geometry,lidar,max_lidar_w,min_lidar_w,min_max_diff,avg_lidar_w
0,1,93,88,93,141,2018,7,22,ca_m_3411934_sw_11_060_20180722_20190209,0,POINT (238343.430 3810848.805),4,5,2,3,3.333333
1,1,85,85,89,137,2018,7,22,ca_m_3411934_sw_11_060_20180722_20190209,0,POINT (238343.547 3810846.093),4,5,2,3,3.333333
2,1,89,85,91,133,2018,7,22,ca_m_3411934_sw_11_060_20180722_20190209,0,POINT (238344.864 3810845.535),4,5,2,3,3.333333
